In [1]:
import numpy as np
import sympy as sym
from scipy import linalg
import json
import matplotlib.pyplot as plt
from ae483tools import *

# 2. Derive models

## 2.1 Define symbolic variables

Define states.

In [2]:
# components of position (meters)
p_x, p_y, p_z = sym.symbols('p_x, p_y, p_z')

# yaw, pitch, and roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi')

# components of linear velocity (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')

# drone position in tracker frame
p_x_t, p_y_t, p_z_t = sym.symbols('p_x^T, p_y^T, p_z^T')

Define inputs.

In [3]:
# gyroscope measurements - components of angular velocity (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# z-axis accelerometer measurement - specific force (meters / second^2)
a_z = sym.symbols('a_z')

Define outputs.

In [4]:
n_x, n_y, r, d = sym.symbols('n_x, n_y, r, d')

Define parameters.

In [5]:
g, k_flow = sym.symbols('g, k_flow')

Create the linear velocity vector $v^B_{W, B}$ and the angular velocity vector $w^B_{W, B}$, both written in the coordinates of the body frame.

In [6]:
v_inB_ofWB = sym.Matrix([v_x, v_y, v_z])
w_inB_ofWB = sym.Matrix([w_x, w_y, w_z])
P_inT_ofB = sym.Matrix([p_x_t, p_y_t, p_z_t])

## 2.2 Define kinematics of orientation

### 2.2.1 Rotation matrix in terms of yaw, pitch, roll angles

Define individual rotation matrices.

In [7]:
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi), sym.cos(psi), 0],
                 [0, 0, 1]])

Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])

Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi), sym.cos(phi)]])

Apply sequential transformation to compute the rotation matrix that describes the orientation of the drone (i.e., of frame $B$ in the coordinates of frame $W$).

In [8]:
R_inW_ofB = Rz * Ry * Rx

### 2.2.2 Map from angular velocity to angular rates

Recall that

$$\begin{bmatrix} \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \end{bmatrix} = N w_{W, B}^{B}$$

for some matrix $N$. Here is how to compute that matrix for a ZYX (yaw, pitch, roll) Euler angle sequence.  First, we compute its inverse:

In [9]:
Ninv = sym.Matrix.hstack((Ry * Rx).T * sym.Matrix([[0], [0], [1]]),
                              (Rx).T * sym.Matrix([[0], [1], [0]]),
                                       sym.Matrix([[1], [0], [0]]))

Then, we compute $N$ by taking the inverse of $N^{-1}$:

In [10]:
N = sym.simplify(Ninv.inv())

## 2.3 Derive equations of motion

Define $a^\text{SF}$, i.e., "acceleration without the gravity term."

In [11]:
aSF = sym.Matrix([
    w_inB_ofWB.cross(v_inB_ofWB)[0],
    w_inB_ofWB.cross(v_inB_ofWB)[1],
    a_z,
])

Define $\dot{P}_B^T$, assuming $v_T^W = 0$

In [13]:
Pdot_inT_ofB = R_inW_ofB * v_inB_ofWB

Create equations of motion.

In [15]:
f = sym.Matrix.vstack(
    R_inW_ofB * v_inB_ofWB,
    N * w_inB_ofWB,
    R_inW_ofB.T * sym.Matrix([0, 0, - g]) + aSF,
    Pdot_inT_ofB
)

Show equations of motion, which have the form

$$\dot{s} = f(s, i, p)$$

where

$$
s = \begin{bmatrix} o_x \\ o_y \\ o_z \\ \psi \\ \theta \\ \phi \\ v_x \\ v_y \\ v_z \\ p_x^T \\ p_y^T \\ p_z^T \end{bmatrix}
\qquad\qquad
i = \begin{bmatrix} w_x \\ w_y \\ w_z \\ a_z \end{bmatrix}
\qquad\qquad
p = \begin{bmatrix} g \\ k_\text{flow} \end{bmatrix}.
$$

In [16]:
f

Matrix([
[ v_x*cos(psi)*cos(theta) + v_y*(sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)) + v_z*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi))],
[v_x*sin(psi)*cos(theta) + v_y*(sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi)) + v_z*(-sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi))],
[                                                                       -v_x*sin(theta) + v_y*sin(phi)*cos(theta) + v_z*cos(phi)*cos(theta)],
[                                                                                         w_y*sin(phi)/cos(theta) + w_z*cos(phi)/cos(theta)],
[                                                                                                               w_y*cos(phi) - w_z*sin(phi)],
[                                                                                   w_x + w_y*sin(phi)*tan(theta) + w_z*cos(phi)*tan(theta)],
[                                                                                                          g*sin(theta) - v_y*w_z + v_z*w_y

## 2.4 Derive measurement equations

Create measurement equations.

In [18]:
h = sym.Matrix([
    k_flow * (v_x - p_z * w_y) / p_z,               # <-- x flow (n_x)
    k_flow * (v_y + p_z * w_x) / p_z,               # <-- y flow (n_y)
    p_z / (sym.cos(phi) * sym.cos(theta)),          # <-- z range (r)
    sym.sqrt(p_x_t ** 2 + p_y_t ** 2 + p_z_t ** 2)  # <-- distance (d)
])

Show measurement equations, which have the form

$$o = h(s, i, p)$$

where

$$
o = \begin{bmatrix} n_x \\ n_y \\ r \\ d\end{bmatrix}
\qquad\qquad
s = \begin{bmatrix} o_x \\ o_y \\ o_z \\ \psi \\ \theta \\ \phi \\ v_x \\ v_y \\ v_z \\ p_x^T \\ p_y^T \\ p_z^T \end{bmatrix}
\qquad\qquad
i = \begin{bmatrix} w_x \\ w_y \\ w_z \\ a_z \end{bmatrix}
\qquad\qquad
p = \begin{bmatrix} g \\ k_\text{flow} \end{bmatrix}.
$$

In [19]:
h

Matrix([
[         k_flow*(-p_z*w_y + v_x)/p_z],
[          k_flow*(p_z*w_x + v_y)/p_z],
[           p_z/(cos(phi)*cos(theta))],
[sqrt(p_x^T**2 + p_y^T**2 + p_z^T**2)]])

# 3. Derive state-space model

## 3.1 Choose equilibrium point

An equilibrium point of the nonlinear system is a choice of states $s_\text{eq}$ and inputs $i_\text{eq}$ — along with constant parameters $p_\text{eq}$ — for which

$$0 = f(s_\text{eq}, i_\text{eq}, p_\text{eq}).$$

Create a symbolic variable to describe the equilibrium value of $p_z$.

In [25]:
p_z_eq = sym.symbols('p_z_eq')
d_eq = sym.symbols('d_eq')

Create a list of states, inputs, outputs, and parameters as symbolic variables.

In [26]:
s = [p_x, p_y, p_z, psi, theta, phi, v_x, v_y, v_z, p_x_t, p_y_t, p_z_t]
i = [w_x, w_y, w_z, a_z]
o = [n_x, n_y, r, d]
p = [g, k_flow]

Create a list of state and input values at equilibrium in the **same order** as before.

In [72]:
s_eq = [0, 0, p_z_eq, 0, 0, 0, 0, 0, 0, d_eq / sym.sqrt(2), d_eq / sym.sqrt(2), 0]
i_eq = [0, 0, 0, g]

Make sure all equilibrium values are symbolic.

In [73]:
s_eq = [sym.nsimplify(a) for a in s_eq]
i_eq = [sym.nsimplify(a) for a in i_eq]

Evaluate the equations of motion at the equilibrium point - if it actually *is* an equilibrium point, then the result should be a matrix of zeros:

In [74]:
f.subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))

Matrix([
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0],
[0]])

## 3.2 Find $A$, $B$, $C$, and $D$

Recall that:

$$
A = \frac{\partial f}{\partial s}\biggr\vert_{(s, i, p) = (s_\text{eq}, i_\text{eq}, p_\text{eq})}
\qquad\quad
B = \frac{\partial f}{\partial i}\biggr\vert_{(s, i, p) = (s_\text{eq}, i_\text{eq}, p_\text{eq})}
\qquad\quad
C = \frac{\partial h}{\partial s}\biggr\vert_{(s, i, p) = (s_\text{eq}, i_\text{eq}, p_\text{eq})}
\qquad\quad
D = \frac{\partial h}{\partial i}\biggr\vert_{(s, i, p) = (s_\text{eq}, i_\text{eq}, p_\text{eq})}.
$$

Compute each Jacobian and plug in the equilibrium values as follows.

In [75]:
A = f.jacobian(s).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
B = f.jacobian(i).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
C = h.jacobian(s).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))
D = h.jacobian(i).subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))

Show $A$:

In [76]:
A

Matrix([
[0, 0, 0, 0, 0,  0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0,  0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, g,  0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, -g, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0,  0, 0, 0, 1, 0, 0, 0]])

Show $B$:

In [77]:
B

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 1, 0],
[0, 1, 0, 0],
[1, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 1],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

Show $C$:

In [78]:
C

Matrix([
[0, 0, 0, 0, 0, 0, k_flow/p_z_eq,             0, 0,                              0,                              0, 0],
[0, 0, 0, 0, 0, 0,             0, k_flow/p_z_eq, 0,                              0,                              0, 0],
[0, 0, 1, 0, 0, 0,             0,             0, 0,                              0,                              0, 0],
[0, 0, 0, 0, 0, 0,             0,             0, 0, sqrt(2)*d_eq/(2*sqrt(d_eq**2)), sqrt(2)*d_eq/(2*sqrt(d_eq**2)), 0]])

Show $D$ (note that it is *not* zero in this case):

In [79]:
D

Matrix([
[     0, -k_flow, 0, 0],
[k_flow,       0, 0, 0],
[     0,       0, 0, 0],
[     0,       0, 0, 0]])

## 3.3 Write linearized models

Define the state, input, and output of the state-space system (i.e., the linearized model of the equations of motion and of the measurement equations).

In [80]:
x = sym.Matrix(s) - sym.Matrix(s_eq)
u = sym.Matrix(i) - sym.Matrix(i_eq)
y = sym.Matrix(o) - h.subs(tuple(zip(s, s_eq))).subs(tuple(zip(i, i_eq)))

Show the linearized equations of motion $Ax+Bu$.

In [81]:
A * x + B * u

Matrix([
[    v_x],
[    v_y],
[    v_z],
[    w_z],
[    w_y],
[    w_x],
[g*theta],
[ -g*phi],
[a_z - g],
[    v_x],
[    v_y],
[    v_z]])

Show the linearized measurement equations $Cx+Du$.

In [82]:
C * x + D * u

Matrix([
[                                                                                    -k_flow*w_y + k_flow*v_x/p_z_eq],
[                                                                                     k_flow*w_x + k_flow*v_y/p_z_eq],
[                                                                                                       p_z - p_z_eq],
[sqrt(2)*d_eq*(-sqrt(2)*d_eq/2 + p_x^T)/(2*sqrt(d_eq**2)) + sqrt(2)*d_eq*(-sqrt(2)*d_eq/2 + p_y^T)/(2*sqrt(d_eq**2))]])

Show the output (which our model tells us should be $Cx+Du$).

In [83]:
y

Matrix([
[              n_x],
[              n_y],
[      -p_z_eq + r],
[d - sqrt(d_eq**2)]])

In [84]:
subs = {g: 9.81, k_flow: 0.01 * 30.0 / np.deg2rad(4.2), p_z_eq: 0.5, d_eq: 1.}

A = np.array(A.subs(subs).evalf()).astype(np.float64)
B = np.array(B.subs(subs).evalf()).astype(np.float64)
C = np.array(C.subs(subs).evalf()).astype(np.float64)
D = np.array(D.subs(subs).evalf()).astype(np.float64)

## 2.2 Show that not all states are observable

Find the observability matrix

$$ W_o = \begin{bmatrix} C \\ CA \\ CA^2 \\ \vdots \\ CA^{n-1} \end{bmatrix} $$

where $A$ is $n \times n$.

In [85]:
def obsv(A, C):
    W = C
    for i in range(1, A.shape[0]):
        W = np.vstack([W, C @ np.linalg.matrix_power(A, i)])
    return W

W_o = obsv(A, C)

Find the rank of the observability matrix using [numpy.linalg.matrix_rank](https://numpy.org/doc/stable/reference/generated/numpy.linalg.matrix_rank.html). The system is **observable** if and only if $W_o$ is **full rank**, that is, if its rank is equal to the number of states $n$.

In [86]:
print(f'      rank of W_o is: {np.linalg.matrix_rank(W_o)}')
print(f'"full rank" would be: {A.shape[0]}')
# print(f"W_o: \n{W_o}\n")
W_str = np.array2string(W_o,
                        formatter={'float_kind': lambda x: f'{x:12.6f}'},
                        prefix='    ',
                        max_line_width=np.inf)
# print(W_str)

      rank of W_o is: 7
"full rank" would be: 12


In [88]:
from scipy.linalg import null_space
ns = null_space(W_o)
ns = ns * np.copysign(1, ns[0, 0])
print(np.array2string(ns, formatter={"float_kind": lambda x: f'{x:12.6f}'},
                prefix='    ',
                max_line_width=np.inf))

[[    0.000000     0.000000     0.000000     0.000000     1.000000]
     [    0.000000    -0.707107    -0.707107     0.000000     0.000000]
     [    0.000000     0.000000     0.000000     0.000000     0.000000]
     [    0.000000     0.000000     0.000000    -1.000000     0.000000]
     [    0.000000    -0.000000    -0.000000     0.000000     0.000000]
     [    0.000000    -0.000000    -0.000000     0.000000     0.000000]
     [    0.000000     0.000000     0.000000     0.000000     0.000000]
     [    0.000000     0.000000     0.000000     0.000000     0.000000]
     [    0.000000     0.000000     0.000000     0.000000     0.000000]
     [    0.000000    -0.500000     0.500000     0.000000     0.000000]
     [    0.000000     0.500000    -0.500000     0.000000     0.000000]
     [    1.000000     0.000000     0.000000     0.000000     0.000000]]


If the system is *not* observable, then it is impossible to design a stable observer - that is, an observer that makes the error in your estimate of each state converge to zero over time.

In particular, the following code would produce an error if you tried to use it:
```python
# Choose weights
Q = np.eye(3) # <-- one diagonal entry for each output
R = np.eye(9) # <-- one diagonal entry for each state

# Find gain matrix
L = lqr(A.T, C.T, linalg.inv(R), linalg.inv(Q)).T
```
It does not matter what method you use - if it is impossible to design a stable observer, that's it, you're out of luck!

## 2.3 Choose a subset of states that are observable

List the index of each state to include.

In [ ]:
s_obs_index = [
    0, # p_x
    1, # p_y
    2, # p_z
    3, # psi
    4, # theta
    5, # phi
    6, # v_x
    7, # v_y
    8, # v_z
]

Define a state-space model

$$
\begin{align*}
\dot{x}_\text{obs} = A_\text{obs} x_\text{obs} + B_\text{obs} u \\
y = C_\text{obs} x_\text{obs} + D_\text{obs} u
\end{align*}
$$

with only these states.

In [ ]:
A_obs = A[s_obs_index, :][:, s_obs_index]
B_obs = B[s_obs_index, :]
C_obs = C[:, s_obs_index]
D_obs = D

Show the matrices that describe this state-space model.

In [ ]:
print(f'A_obs:\n{A_obs}\n')
print(f'B_obs:\n{B_obs}\n')
print(f'C_obs:\n{C_obs}\n')
print(f'D_obs:\n{D_obs}\n')

A_obs:
[[ 0.    0.    0.    0.    0.    0.    1.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    1.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    1.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    9.81  0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.   -9.81  0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.  ]]

B_obs:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]]

C_obs:
[[0.         0.         0.         0.         0.         0.
  8.18511136 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         8.18511136 0.        ]
 [0.         0.         1.         0.         0.         0.
  0.         0.         0.        ]
 [1.         0.         0.         0.  

Check that this subsystem is observable:

In [ ]:
print(f'      rank of W_o is: {np.linalg.matrix_rank(obsv(A_obs, C_obs))}')
print(f'"full rank" would be: {A_obs.shape[0]}')

      rank of W_o is: 9
"full rank" would be: 9


## 2.4 Choose gain matrix for the observable subsystem

### 2.4.1 With equal weights

Choose weights $Q$ and $R$ as identity matrices.

In [ ]:
# FIXME: change the size of each identity matrix to match the
# number of states and outputs for your observable subsystem

Q = np.eye(9) # <-- one diagonal entry for each output
R = np.eye(9) # <-- one diagonal entry for each state

Find gain matrix $L$ for the chosen $Q$ and $R$ by solving an LQR problem.

In [ ]:
L = lqr(A_obs.T, C_obs.T, linalg.inv(R), linalg.inv(Q)).T

Show $L$ (formatted nicely).

In [ ]:
L_str = np.array2string(L,
                        formatter={'float_kind': lambda x: f'{x:12.6f}'},
                        prefix='    ',
                        max_line_width=np.inf)

print(f'L = {L_str}')

L = [[    0.120659     0.000000     0.000000     1.007434    -0.000000     0.000000    -0.000000     0.001247     0.000000]
     [   -0.000000     0.120659    -0.000000    -0.000000     1.007434    -0.000000     0.000000    -0.000000    -0.001247]
     [   -0.000000    -0.000000     1.098684     0.000000    -0.000000     1.098684     0.000000    -0.000000     0.000000]
     [   -0.000000    -0.000000     0.000000    -0.000000     0.000000     0.000000     1.000000    -0.000000    -0.000000]
     [    0.982670    -0.000000    -0.000000     0.001247    -0.000000    -0.000000    -0.000000     0.185362     0.000000]
     [    0.000000    -0.982670     0.000000     0.000000    -0.001247     0.000000    -0.000000     0.000000     0.185362]
     [    1.827803    -0.000000    -0.000000     0.014741    -0.000000    -0.000000    -0.000000     0.120056     0.000000]
     [   -0.000000     1.827803    -0.000000     0.000000     0.014741    -0.000000    -0.000000    -0.000000    -0.120056]
     [  